# Notebook to generate sample data from a postgis demoserver

Loads sample tables from a PostGIS demoserver. Illustrating to different methods: DuckDB and GeoPandas. 

Requires a .env with
```
pgcreds=postgresql://...
```

Tables are:
```
tables = [
    "arealbruk_skogbonitet_sample",
    "buildings_sample",
    "flomsoner_sample",
    "fotrute_senterlinje",
    "fotruteinfo_tabell",
    "skiloype_senterlinje",
    "skiloypeinfo_tabell",
    "sykkelrute_senterlinje_sample",
]
```

# Export PostGIS to DuckDB and GeoParquet

This notebook reads tables from PostGIS using credentials in a local `.env` file (`pgcreds=postgresql://...`), writes a DuckDB file with spatial tables, and exports GeoParquet files.

Required packages (install in your environment if missing):
- geopandas
- duckdb
- python-dotenv
- sqlalchemy
- psycopg2-binary
- pyarrow

In [4]:
import subprocess

subprocess.run([
    "uv", "pip", "install",
    "geopandas",
    "duckdb",
    "python-dotenv",
    "sqlalchemy",
    "psycopg2-binary",
    "pyarrow"
])

Using Python 3.12.12 environment at: /Users/alexandesn/dev/.virtualenvs/geoenv
Resolved 16 packages in 713ms
 Downloaded sqlalchemy
 Downloaded psycopg2-binary
Prepared 3 packages in 467ms
Installed 3 packages in 6ms
 + psycopg2-binary==2.9.11
 + python-dotenv==1.2.1
 + sqlalchemy==2.0.47


CompletedProcess(args=['uv', 'pip', 'install', 'geopandas', 'duckdb', 'python-dotenv', 'sqlalchemy', 'psycopg2-binary', 'pyarrow'], returncode=0)

In [6]:
from __future__ import annotations

import os
from pathlib import Path

import duckdb
from dotenv import load_dotenv

# Load local .env with pgcreds=postgresql://...
load_dotenv()
pgcreds = os.environ.get("pgcreds")
if not pgcreds:
    raise ValueError("Missing pgcreds in .env")

output_dir = Path("outputs")
output_dir.mkdir(parents=True, exist_ok=True)

duckdb_path = output_dir / "postgis_dump.duckdb"
con = duckdb.connect(str(duckdb_path))

# Install and load extensions
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")
con.execute("INSTALL postgres;")
con.execute("LOAD postgres;")

tables = [
    "arealbruk_skogbonitet_sample",
    "buildings_sample",
    "flomsoner_sample",
    "fotrute_senterlinje",
    "fotruteinfo_tabell",
    "skiloype_senterlinje",
    "skiloypeinfo_tabell",
    "sykkelrute_senterlinje_sample",
]

# Attach to the postgres database
con.execute(f"ATTACH '{pgcreds}' AS pg (TYPE POSTGRES);")

# Copy each table from postgres to duckdb
for table in tables:
    print(f"Copying {table}...")
    con.execute(f"CREATE TABLE {table} AS SELECT * FROM pg.public.{table};")

con.close()
print(f"DuckDB with all tables saved to: {duckdb_path}")

Copying arealbruk_skogbonitet_sample...
Copying buildings_sample...
Copying flomsoner_sample...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Copying fotrute_senterlinje...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Copying fotruteinfo_tabell...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Copying skiloype_senterlinje...
Copying skiloypeinfo_tabell...
Copying sykkelrute_senterlinje_sample...
DuckDB with all tables saved to: outputs/postgis_dump.duckdb


In [7]:
from __future__ import annotations

import os
from pathlib import Path

import geopandas as gpd
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

# Load local .env with pgcreds=postgresql://...
load_dotenv()
pgcreds = os.environ.get("pgcreds")
if not pgcreds:
    raise ValueError("Missing pgcreds in .env")

engine = create_engine(pgcreds)

tables = [
    "arealbruk_skogbonitet_sample",
    "buildings_sample",
    "flomsoner_sample",
    "fotrute_senterlinje",
    "fotruteinfo_tabell",
    "skiloype_senterlinje",
    "skiloypeinfo_tabell",
    "sykkelrute_senterlinje_sample",
]

output_dir = Path("outputs")
geoparquet_dir = output_dir / "geoparquet"
parquet_dir = output_dir / "parquet"
geoparquet_dir.mkdir(parents=True, exist_ok=True)
parquet_dir.mkdir(exist_ok=True)


def get_geometry_column(table: str) -> str | None:
    """Find the geometry column in the table."""
    query = text(
        """
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema = 'public'
          AND table_name = :table
          AND udt_name = 'geometry'
        LIMIT 1
        """
    )
    with engine.connect() as conn:
        row = conn.execute(query, {"table": table}).fetchone()
    return row[0] if row else None


def quote_ident(name: str) -> str:
    """Quote identifier for SQL."""
    return '"' + name.replace('"', '""') + '"'


# Process each table
for table in tables:
    geom_col = get_geometry_column(table)
    table_sql = quote_ident(table)
    print(f"Processing {table}...")

    if geom_col:
        # Read spatial table
        gdf = gpd.read_postgis(f"SELECT * FROM {table_sql}", engine, geom_col=geom_col)
        geoparquet_path = geoparquet_dir / f"{table}.parquet"
        gdf.to_parquet(geoparquet_path, index=False)
        print(f"  Saved to: {geoparquet_path}")
    else:
        # Read non-spatial table
        df = pd.read_sql(f"SELECT * FROM {table_sql}", engine)
        parquet_path = parquet_dir / f"{table}.parquet"
        df.to_parquet(parquet_path, index=False)
        print(f"  Saved to: {parquet_path}")

print(f"\nGeoParquet files saved to: {geoparquet_dir}")
print(f"Parquet files saved to: {parquet_dir}")

Processing arealbruk_skogbonitet_sample...
  Saved to: outputs/geoparquet/arealbruk_skogbonitet_sample.parquet
Processing buildings_sample...
  Saved to: outputs/geoparquet/buildings_sample.parquet
Processing flomsoner_sample...
  Saved to: outputs/geoparquet/flomsoner_sample.parquet
Processing fotrute_senterlinje...
  Saved to: outputs/geoparquet/fotrute_senterlinje.parquet
Processing fotruteinfo_tabell...
  Saved to: outputs/parquet/fotruteinfo_tabell.parquet
Processing skiloype_senterlinje...
  Saved to: outputs/geoparquet/skiloype_senterlinje.parquet
Processing skiloypeinfo_tabell...
  Saved to: outputs/parquet/skiloypeinfo_tabell.parquet
Processing sykkelrute_senterlinje_sample...
  Saved to: outputs/geoparquet/sykkelrute_senterlinje_sample.parquet

GeoParquet files saved to: outputs/geoparquet
Parquet files saved to: outputs/parquet
